In [30]:
import librosa 
import numpy as np
def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 
     
    return mfccsscaled

In [31]:
# Load various imports 
import pandas as pd
import os
import librosa

# Set the path to the full UrbanSound dataset 
fulldatasetpath = '../dataset/audio/'

#metadata = pd.read_csv('../dataset/dataset.csv')
metadata = pd.read_csv('../dataset/datasetM2.csv')

features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():
    
    file_name = os.path.join(os.path.abspath(fulldatasetpath),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    
    class_label = row["class_name"]
    data = extract_features(file_name)
    
    features.append([data, class_label])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files') 

Finished feature extraction from  207  files


In [32]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

In [33]:
# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [34]:
### store the preprocessed data for use in the next notebook

%store x_train 
%store x_test 
%store y_train 
%store y_test 
%store yy 
%store le

Stored 'x_train' (ndarray)
Stored 'x_test' (ndarray)
Stored 'y_train' (ndarray)
Stored 'y_test' (ndarray)
Stored 'yy' (ndarray)
Stored 'le' (LabelEncoder)


In [35]:
# retrieve the preprocessed data from previous notebook

%store -r x_train 
%store -r x_test 
%store -r y_train 
%store -r y_test 
%store -r yy 
%store -r le

In [36]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [37]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

In [38]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               10496     
_________________________________________________________________
activation_4 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_5 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 5)                

In [39]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

num_epochs = 350
num_batch_size = 40

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 165 samples, validate on 42 samples
Epoch 1/350
165/165 [==============================] - 0s 982us/step - loss: 60.2136 - accuracy: 0.3273 - val_loss: 30.9474 - val_accuracy: 0.4524

Epoch 00001: val_loss improved from inf to 30.94742, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 2/350
165/165 [==============================] - 0s 81us/step - loss: 49.9171 - accuracy: 0.3697 - val_loss: 37.3315 - val_accuracy: 0.4524

Epoch 00002: val_loss did not improve from 30.94742
Epoch 3/350
165/165 [==============================] - 0s 72us/step - loss: 48.6894 - accuracy: 0.3455 - val_loss: 12.2164 - val_accuracy: 0.4524

Epoch 00003: val_loss improved from 30.94742 to 12.21641, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 4/350
165/165 [==============================] - 0s 70us/step - loss: 52.4435 - accuracy: 0.2606 - val_loss: 10.0927 - val_accuracy: 0.4524

Epoch 00004: val_loss improved from 12.21641 to 10.09269, saving model to saved_models/we

165/165 [==============================] - 0s 60us/step - loss: 3.1546 - accuracy: 0.4364 - val_loss: 1.0360 - val_accuracy: 0.4048

Epoch 00037: val_loss did not improve from 0.99622
Epoch 38/350
165/165 [==============================] - 0s 61us/step - loss: 3.2020 - accuracy: 0.4727 - val_loss: 1.0385 - val_accuracy: 0.4048

Epoch 00038: val_loss did not improve from 0.99622
Epoch 39/350
165/165 [==============================] - 0s 58us/step - loss: 2.4983 - accuracy: 0.4121 - val_loss: 1.0405 - val_accuracy: 0.4048

Epoch 00039: val_loss did not improve from 0.99622
Epoch 40/350
165/165 [==============================] - 0s 59us/step - loss: 2.4900 - accuracy: 0.4606 - val_loss: 1.0433 - val_accuracy: 0.4048

Epoch 00040: val_loss did not improve from 0.99622
Epoch 41/350
165/165 [==============================] - 0s 58us/step - loss: 2.3496 - accuracy: 0.4303 - val_loss: 1.0488 - val_accuracy: 0.4048

Epoch 00041: val_loss did not improve from 0.99622
Epoch 42/350
165/165 [======


Epoch 00078: val_loss did not improve from 0.99622
Epoch 79/350
165/165 [==============================] - 0s 75us/step - loss: 1.2536 - accuracy: 0.4970 - val_loss: 1.0400 - val_accuracy: 0.4048

Epoch 00079: val_loss did not improve from 0.99622
Epoch 80/350
165/165 [==============================] - 0s 62us/step - loss: 1.3144 - accuracy: 0.4848 - val_loss: 1.0392 - val_accuracy: 0.4048

Epoch 00080: val_loss did not improve from 0.99622
Epoch 81/350
165/165 [==============================] - 0s 64us/step - loss: 1.3510 - accuracy: 0.4424 - val_loss: 1.0386 - val_accuracy: 0.4286

Epoch 00081: val_loss did not improve from 0.99622
Epoch 82/350
165/165 [==============================] - 0s 60us/step - loss: 1.5453 - accuracy: 0.4909 - val_loss: 1.0382 - val_accuracy: 0.4286

Epoch 00082: val_loss did not improve from 0.99622
Epoch 83/350
165/165 [==============================] - 0s 60us/step - loss: 1.1669 - accuracy: 0.4848 - val_loss: 1.0384 - val_accuracy: 0.4286

Epoch 00083: v

165/165 [==============================] - 0s 60us/step - loss: 1.1349 - accuracy: 0.5091 - val_loss: 1.0390 - val_accuracy: 0.4524

Epoch 00120: val_loss did not improve from 0.99622
Epoch 121/350
165/165 [==============================] - 0s 57us/step - loss: 1.1352 - accuracy: 0.5212 - val_loss: 1.0383 - val_accuracy: 0.4524

Epoch 00121: val_loss did not improve from 0.99622
Epoch 122/350
165/165 [==============================] - 0s 57us/step - loss: 1.2673 - accuracy: 0.4727 - val_loss: 1.0363 - val_accuracy: 0.4762

Epoch 00122: val_loss did not improve from 0.99622
Epoch 123/350
165/165 [==============================] - 0s 59us/step - loss: 1.1317 - accuracy: 0.4788 - val_loss: 1.0349 - val_accuracy: 0.5000

Epoch 00123: val_loss did not improve from 0.99622
Epoch 124/350
165/165 [==============================] - 0s 61us/step - loss: 1.1152 - accuracy: 0.4788 - val_loss: 1.0345 - val_accuracy: 0.4762

Epoch 00124: val_loss did not improve from 0.99622
Epoch 125/350
165/165 [=

165/165 [==============================] - 0s 77us/step - loss: 1.1211 - accuracy: 0.5212 - val_loss: 1.0254 - val_accuracy: 0.4524

Epoch 00161: val_loss did not improve from 0.99622
Epoch 162/350
165/165 [==============================] - 0s 63us/step - loss: 1.1255 - accuracy: 0.4970 - val_loss: 1.0268 - val_accuracy: 0.5000

Epoch 00162: val_loss did not improve from 0.99622
Epoch 163/350
165/165 [==============================] - 0s 60us/step - loss: 1.1475 - accuracy: 0.4848 - val_loss: 1.0286 - val_accuracy: 0.4762

Epoch 00163: val_loss did not improve from 0.99622
Epoch 164/350
165/165 [==============================] - 0s 58us/step - loss: 1.1307 - accuracy: 0.4909 - val_loss: 1.0313 - val_accuracy: 0.4762

Epoch 00164: val_loss did not improve from 0.99622
Epoch 165/350
165/165 [==============================] - 0s 57us/step - loss: 1.1464 - accuracy: 0.4606 - val_loss: 1.0312 - val_accuracy: 0.4762

Epoch 00165: val_loss did not improve from 0.99622
Epoch 166/350
165/165 [=

165/165 [==============================] - 0s 62us/step - loss: 1.0924 - accuracy: 0.5455 - val_loss: 1.0083 - val_accuracy: 0.4524

Epoch 00202: val_loss did not improve from 0.99622
Epoch 203/350
165/165 [==============================] - 0s 57us/step - loss: 1.1023 - accuracy: 0.5515 - val_loss: 1.0061 - val_accuracy: 0.4524

Epoch 00203: val_loss did not improve from 0.99622
Epoch 204/350
165/165 [==============================] - 0s 62us/step - loss: 1.0914 - accuracy: 0.5273 - val_loss: 1.0042 - val_accuracy: 0.4524

Epoch 00204: val_loss did not improve from 0.99622
Epoch 205/350
165/165 [==============================] - 0s 59us/step - loss: 1.1424 - accuracy: 0.4848 - val_loss: 1.0035 - val_accuracy: 0.4524

Epoch 00205: val_loss did not improve from 0.99622
Epoch 206/350
165/165 [==============================] - 0s 54us/step - loss: 1.1468 - accuracy: 0.5273 - val_loss: 1.0037 - val_accuracy: 0.4524

Epoch 00206: val_loss did not improve from 0.99622
Epoch 207/350
165/165 [=

165/165 [==============================] - 0s 57us/step - loss: 1.0028 - accuracy: 0.5879 - val_loss: 0.9313 - val_accuracy: 0.5000

Epoch 00239: val_loss improved from 0.93165 to 0.93132, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 240/350
165/165 [==============================] - 0s 55us/step - loss: 1.0551 - accuracy: 0.5455 - val_loss: 0.9255 - val_accuracy: 0.5000

Epoch 00240: val_loss improved from 0.93132 to 0.92553, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 241/350
165/165 [==============================] - 0s 56us/step - loss: 1.0486 - accuracy: 0.5333 - val_loss: 0.9223 - val_accuracy: 0.5000

Epoch 00241: val_loss improved from 0.92553 to 0.92233, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 242/350
165/165 [==============================] - 0s 61us/step - loss: 1.0715 - accuracy: 0.5212 - val_loss: 0.9213 - val_accuracy: 0.5000

Epoch 00242: val_loss improved from 0.92233 to 0.92128, saving model to saved_models/we

Epoch 274/350
165/165 [==============================] - 0s 61us/step - loss: 0.9047 - accuracy: 0.6485 - val_loss: 0.8069 - val_accuracy: 0.6429

Epoch 00274: val_loss did not improve from 0.80178
Epoch 275/350
165/165 [==============================] - 0s 60us/step - loss: 0.9403 - accuracy: 0.6000 - val_loss: 0.7929 - val_accuracy: 0.6429

Epoch 00275: val_loss improved from 0.80178 to 0.79292, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 276/350
165/165 [==============================] - 0s 59us/step - loss: 0.9717 - accuracy: 0.6000 - val_loss: 0.7843 - val_accuracy: 0.6667

Epoch 00276: val_loss improved from 0.79292 to 0.78430, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 277/350
165/165 [==============================] - 0s 63us/step - loss: 0.8889 - accuracy: 0.6242 - val_loss: 0.7828 - val_accuracy: 0.6190

Epoch 00277: val_loss improved from 0.78430 to 0.78282, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 278/350
165/165 

165/165 [==============================] - 0s 60us/step - loss: 0.8572 - accuracy: 0.6242 - val_loss: 0.7081 - val_accuracy: 0.6429

Epoch 00311: val_loss did not improve from 0.68373
Epoch 312/350
165/165 [==============================] - 0s 58us/step - loss: 0.7873 - accuracy: 0.6485 - val_loss: 0.7053 - val_accuracy: 0.6905

Epoch 00312: val_loss did not improve from 0.68373
Epoch 313/350
165/165 [==============================] - 0s 69us/step - loss: 0.8646 - accuracy: 0.6061 - val_loss: 0.7063 - val_accuracy: 0.6905

Epoch 00313: val_loss did not improve from 0.68373
Epoch 314/350
165/165 [==============================] - 0s 61us/step - loss: 0.8253 - accuracy: 0.6121 - val_loss: 0.6997 - val_accuracy: 0.7143

Epoch 00314: val_loss did not improve from 0.68373
Epoch 315/350
165/165 [==============================] - 0s 56us/step - loss: 0.8174 - accuracy: 0.6303 - val_loss: 0.6973 - val_accuracy: 0.6905

Epoch 00315: val_loss did not improve from 0.68373
Epoch 316/350
165/165 [=


Epoch 00349: val_loss did not improve from 0.62072
Epoch 350/350
165/165 [==============================] - 0s 57us/step - loss: 0.7790 - accuracy: 0.6788 - val_loss: 0.6400 - val_accuracy: 0.6905

Epoch 00350: val_loss did not improve from 0.62072
Training completed in time:  0:00:06.451447


In [40]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.7333333492279053
Testing Accuracy:  0.6904761791229248


In [41]:
import librosa 
import numpy as np 

def extract_feature(file_name):
   
    try:
        audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None, None

    return np.array([mfccsscaled])


In [42]:
def print_prediction(file_name):
    prediction_feature = extract_feature(file_name) 

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("The predicted class is:", predicted_class[0], '\n') 

    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )

In [56]:
# test

filename = '../dataset/audio/ebtest.wav' 
print_prediction(filename) 

The predicted class is: tap 

double_tap 		 :  0.40343210101127624511718750000000
long_tap 		 :  0.00316147739067673683166503906250
swipe 		 :  0.07973125576972961425781250000000
tap 		 :  0.50079238414764404296875000000000
zoom 		 :  0.01288282033056020736694335937500
